In [13]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
batch_size =64
epochs=100
latent_dim=256
min_samples=10000
data_path='fra.txt'

In [14]:
#vectorizng Data
input_texts=[]
target_texts=[]
input_chars=set()
target_chars=set()
with open(data_path,'r',encoding='utf-8') as f:
    lines=f.read().split('\n')
for lines in lines[: min(min_samples,len(lines)-1)]:
    input_text,target_text,_=lines.split('\t')
    target_text='\t'+target_text+'\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_chars:
            input_chars.add(char)
    for char in target_text:
        if char not in target_chars:
            target_chars.add(char)
    
    

In [15]:
input_chars=sorted(list(input_chars))
target_chars=sorted(list(target_chars))
num_encoder_tokens=len(input_chars)
num_decoder_tokens=len(target_chars)
max_encoder_seq_length=max([len(txt) for txt in input_texts])
max_decoder_seq_length=max([len(txt) for txt in target_texts])


In [16]:
print('Number of Samples:',len(input_texts))
print("Number of unique input tokens:",num_encoder_tokens)
print("Number of unique output tokens:",num_decoder_tokens)
print('Max sequence length for inputs:',max_encoder_seq_length)
print('Max sequence length for outputs:',max_decoder_seq_length)


Number of Samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 92
Max sequence length for inputs: 15
Max sequence length for outputs: 59


In [17]:
input_token_index=dict(
[(char,i) for i , char in enumerate(input_chars)])
target_token_index=dict(
[(char,i) for i , char in enumerate(target_chars)])


In [18]:
input_token_index,target_token_index

({' ': 0,
  '!': 1,
  '"': 2,
  '$': 3,
  '%': 4,
  '&': 5,
  "'": 6,
  ',': 7,
  '-': 8,
  '.': 9,
  '0': 10,
  '1': 11,
  '2': 12,
  '3': 13,
  '5': 14,
  '7': 15,
  '8': 16,
  '9': 17,
  ':': 18,
  '?': 19,
  'A': 20,
  'B': 21,
  'C': 22,
  'D': 23,
  'E': 24,
  'F': 25,
  'G': 26,
  'H': 27,
  'I': 28,
  'J': 29,
  'K': 30,
  'L': 31,
  'M': 32,
  'N': 33,
  'O': 34,
  'P': 35,
  'Q': 36,
  'R': 37,
  'S': 38,
  'T': 39,
  'U': 40,
  'V': 41,
  'W': 42,
  'Y': 43,
  'a': 44,
  'b': 45,
  'c': 46,
  'd': 47,
  'e': 48,
  'f': 49,
  'g': 50,
  'h': 51,
  'i': 52,
  'j': 53,
  'k': 54,
  'l': 55,
  'm': 56,
  'n': 57,
  'o': 58,
  'p': 59,
  'q': 60,
  'r': 61,
  's': 62,
  't': 63,
  'u': 64,
  'v': 65,
  'w': 66,
  'x': 67,
  'y': 68,
  'z': 69,
  'é': 70},
 {'\t': 0,
  '\n': 1,
  ' ': 2,
  '!': 3,
  '%': 4,
  '&': 5,
  "'": 6,
  '(': 7,
  ')': 8,
  ',': 9,
  '-': 10,
  '.': 11,
  '0': 12,
  '1': 13,
  '2': 14,
  '3': 15,
  '5': 16,
  '8': 17,
  '9': 18,
  ':': 19,
  '?': 20,
  'A'

In [19]:
encoder_input_data=np.zeros((len(input_texts),max_encoder_seq_length,num_encoder_tokens), dtype='float32')
decoder_input_data=np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_tokens), dtype='float32')
decoder_target_data=np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_tokens), dtype='float32')

In [20]:
#one hot representation
for i ,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    for t ,char in enumerate(input_text):
        encoder_input_data[i,t,input_token_index[char]]=1
    encoder_input_data[i,t+1:,input_token_index[char]]=1
    for t, char in enumerate(target_text):
        decoder_input_data[i,t,target_token_index[char]]=1
        if t>0:
            decoder_target_data[i,t-1,target_token_index[char]]=1
    decoder_input_data[i,t+1:,target_token_index[' ']]=1
    decoder_target_data[i,t:,target_token_index[' ']]=1

In [21]:
encoder_input_data[0].shape

(15, 71)

In [25]:
input_text

'I can hear you.'

In [22]:
encoder_inputs=Input(shape=(None,num_encoder_tokens))
encoder=LSTM(latent_dim,return_state=True)
encoder_outputs,state_h,state_c=encoder(encoder_inputs)
encoder_states=[state_h,state_c]

In [23]:
decoder_inputs=Input(shape=(None,num_decoder_tokens))
decoder_lstm=LSTM(latent_dim,return_sequences=True,return_state=True)
decoder_outputs,_, _=decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense=Dense(num_decoder_tokens,activation='softmax')
decoder_outputs=decoder_dense(decoder_outputs)

In [24]:
model=Model([encoder_inputs,decoder_inputs],decoder_outputs)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit([encoder_input_data,decoder_input_data],decoder_target_data,batch_size =batch_size,epochs=epochs,validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 25s 177ms/step - loss: 1.4888 - accuracy: 0.7070 - val_loss: 1.0206 - val_accuracy: 0.7145
Epoch 2/100
125/125 [==============================] - 23s 187ms/step - loss: 0.8729 - accuracy: 0.7634 - val_loss: 0.8297 - val_accuracy: 0.7710
Epoch 3/100
125/125 [==============================] - 22s 177ms/step - loss: 0.6713 - accuracy: 0.8128 - val_loss: 0.6848 - val_accuracy: 0.8046
Epoch 4/100
125/125 [==============================] - 23s 187ms/step - loss: 0.5780 - accuracy: 0.8329 - val_loss: 0.6208 - val_accuracy: 0.8215
Epoch 5/100
125/125 [==============================] - 26s 209ms/step - loss: 0.5234 - accuracy: 0.8474 - val_loss: 0.5822 - val_accuracy: 0.8317
Epoch 6/100
125/125 [==============================] - 26s 206ms/step - loss: 0.4857 - accuracy: 0.8574 - val_loss: 0.5495 - val_accuracy: 0.8399
Epoch 7/100
125/125 [==============================] - 23s 183ms/step - loss: 0.4574 - accuracy: 0.8653 - val_loss: 0.5306 -

Epoch 57/100
125/125 [==============================] - 22s 180ms/step - loss: 0.0786 - accuracy: 0.9752 - val_loss: 0.6244 - val_accuracy: 0.8721
Epoch 58/100
125/125 [==============================] - 29s 235ms/step - loss: 0.0768 - accuracy: 0.9755 - val_loss: 0.6287 - val_accuracy: 0.8709
Epoch 59/100
125/125 [==============================] - 28s 220ms/step - loss: 0.0744 - accuracy: 0.9764 - val_loss: 0.6323 - val_accuracy: 0.8716
Epoch 60/100
125/125 [==============================] - 28s 226ms/step - loss: 0.0724 - accuracy: 0.9771 - val_loss: 0.6454 - val_accuracy: 0.8701
Epoch 61/100
125/125 [==============================] - 28s 224ms/step - loss: 0.0703 - accuracy: 0.9778 - val_loss: 0.6566 - val_accuracy: 0.8702
Epoch 62/100
125/125 [==============================] - 28s 223ms/step - loss: 0.0693 - accuracy: 0.9781 - val_loss: 0.6537 - val_accuracy: 0.8696
Epoch 63/100
125/125 [==============================] - 27s 212ms/step - loss: 0.0669 - accuracy: 0.9787 - val_loss: 0

In [ ]:
encoder_model=Model(encoder_inputs,encoder_states)
decoder_state_input_h=Input(shape=(latent_dim,))
decoder_state_input_c=Input(shape=(latent_dim,))
decoder_states_inputs=[decoder_state_input_h,decoder_state_input_c]
decoder_ouputs,state_h,state_c=decoder_lstm(decoder_inputs,initial_state=decoder_states_inputs)
decoder_states=[state_h,state_c]
decoder_ouputs=decoder_dense(decoder_outputs)
decoder_model=Model([decoder_inputs]+decoder_states_inputs,[decoder_outputs]+decoder_states)

reverse_input_char_index=dict((i,char) for char,i in input_token_index.items())
reverse_target_char_index=dict((i,char) for char,i in target_token_index.items())

def decode_sequence(input_seq):
    states_value=encoder_model.predict(input_seq)
    
    target_seq =np.zeros((1,1,num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']]=1.
    
    stop_condition=False
    decoded_sentence=''
    while not stop_condition:
        output_tokens, h, c=decoder_model.predict(
        [target_seq]+states_value)
        
        sampled_token_index=np.argmax(output_token[0,-1, :])
        sampled_char=reverse_target_char_index[sampled_token_index]
        decoded_sentence+=sampled_char
        
        #Exit Condition:either hit max length or find stop charater
        if(sampled_char== '\n' or len(decoded_sentence)>max_decoder_seq_length):
            stop_condition=True
        #update the target sequence(of length 1)
        target_seq=np.zeros((1,1,num_decoder_tokens))
        target_seq[0,0,sampled_token_index]=1
        
        #update states
        states_value=[h,c]
    return decoded_sentence
for seq_index in range(100):
    input_seq=encoder_input_data[seq_index: seq_index+1]
    decoded_sentence=decode_sequence(input_seq)
    print('-')
    print('Input sentence:',input_texts[seq_index])
    print('Decoded sentence:',decoded_sentence)
        
    